In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os,sys
import shutil
from PIL import Image

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import *
from keras.layers import *
from keras.regularizers import *
from keras.optimizers import *

#from IPython.display import Image
from keras import applications

C:\Users\vikyl\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Here, we try to resize all the images to 100x100 pixel size, we aren't reducing the size since that might leads to information loss.

In [2]:
path = ("handgesturedata/")
if not os.path.exists("preview"):
    os.makedirs("preview")
img_width, img_height = 100, 100
batchsize=32
for subfolder in os.listdir(path):
    print(subfolder)
    for item in os.listdir(path+subfolder):
        #print(item)
        if item.endswith(".bmp"):
            im = Image.open(path+subfolder+"/"+item)
            print(path+subfolder+"/"+item)
            f, e = os.path.split(path+subfolder+"/"+item)
            imResize = im.resize((100,100), Image.ANTIALIAS)
            im.close()
            imResize.save(path+subfolder+"/"+item,"JPEG")

A
handgesturedata/A/a01.bmp
handgesturedata/A/a02.bmp
handgesturedata/A/a03.bmp
handgesturedata/A/a04.bmp
handgesturedata/A/a05.bmp
handgesturedata/A/a06.bmp
handgesturedata/A/a07.bmp
handgesturedata/A/a08.bmp
handgesturedata/A/a09.bmp
handgesturedata/A/a10.bmp
handgesturedata/A/a11.bmp
handgesturedata/A/a12.bmp
handgesturedata/A/a13.bmp
handgesturedata/A/a14.bmp
handgesturedata/A/a15.bmp
handgesturedata/A/a16.bmp
handgesturedata/A/a17.bmp
handgesturedata/A/a18.bmp
B
handgesturedata/B/b01.bmp
handgesturedata/B/b02.bmp
handgesturedata/B/b03.bmp
handgesturedata/B/b04.bmp
handgesturedata/B/b05.bmp
handgesturedata/B/b06.bmp
handgesturedata/B/b07.bmp
handgesturedata/B/b08.bmp
handgesturedata/B/b09.bmp
handgesturedata/B/b10.bmp
handgesturedata/B/b11.bmp
handgesturedata/B/b12.bmp
handgesturedata/B/b13.bmp
handgesturedata/B/b14.bmp
handgesturedata/B/b15.bmp
handgesturedata/B/b16.bmp
handgesturedata/B/b17.bmp
handgesturedata/B/b18.bmp
handgesturedata/B/b19.bmp
C
handgesturedata/C/c01.bmp
handge

Now we divide the dataset into training and validation set for training a convolutional neural network

In [3]:
for subfolder in os.listdir(path):
    print(subfolder)
    trainingset = os.listdir(path+subfolder)[:2*len(os.listdir(path+subfolder))//3]
    validationset = os.listdir(path+subfolder)[2*len(os.listdir(path+subfolder))//3:]
    print(trainingset)
    print(validationset)
    if not os.path.exists("data/train/"+subfolder):
        os.makedirs("data/train/"+subfolder)
        os.makedirs("data/validation/"+subfolder)
        
    for item in trainingset:
        shutil.copy2(path+subfolder+'/'+item,"data/train/"+subfolder)
        
    for item in validationset:
        shutil.copy2(path+subfolder+'/'+item,"data/validation/"+subfolder)

A
['a01.bmp', 'a02.bmp', 'a03.bmp', 'a04.bmp', 'a05.bmp', 'a06.bmp', 'a07.bmp', 'a08.bmp', 'a09.bmp', 'a10.bmp', 'a11.bmp', 'a12.bmp']
['a13.bmp', 'a14.bmp', 'a15.bmp', 'a16.bmp', 'a17.bmp', 'a18.bmp']
B
['b01.bmp', 'b02.bmp', 'b03.bmp', 'b04.bmp', 'b05.bmp', 'b06.bmp', 'b07.bmp', 'b08.bmp', 'b09.bmp', 'b10.bmp', 'b11.bmp', 'b12.bmp']
['b13.bmp', 'b14.bmp', 'b15.bmp', 'b16.bmp', 'b17.bmp', 'b18.bmp', 'b19.bmp']
C
['c01.bmp', 'c02.bmp', 'c03.bmp', 'c04.bmp', 'c05.bmp', 'c06.bmp', 'c07.bmp', 'c08.bmp', 'c09.bmp']
['c10.bmp', 'c11.bmp', 'c12.bmp', 'c13.bmp', 'c14.bmp']
D
['d01.bmp', 'd02.bmp', 'd03.bmp', 'd04.bmp', 'd05.bmp', 'd06.bmp', 'd07.bmp', 'd08.bmp', 'd09.bmp', 'd10.bmp', 'd11.bmp']
['d12.bmp', 'd13.bmp', 'd14.bmp', 'd15.bmp', 'd16.bmp', 'd17.bmp']
NA
['1.bmp', '10.bmp', '11.bmp', '12.bmp', '13.bmp', '14.bmp', '15.bmp', '16.bmp', '17.bmp', '2.bmp', '3.bmp']
['4.bmp', '5.bmp', '6.bmp', '7.bmp', '8.bmp', '9.bmp']


In [4]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
train_generator = ImageDataGenerator(width_shift_range = 0.1,height_shift_range = 0.1, horizontal_flip=True,shear_range=0.1)

train = train_generator.flow_from_directory("data/train",target_size=(img_height,img_width),batch_size=32,class_mode='categorical')

"""
plt.figure(figsize=(10,10))
for image in enumerate(train_generator):
    plt.subplot(6,6)
    plt.imshow(image)
"""

img=load_img("data/train/A/a01.bmp")
x=img_to_array(img)
x=x.reshape((1,)+x.shape)
print(x.shape)
i=0
for batch in train_generator.flow(x,batch_size=1, save_to_dir='preview', save_prefix='preview', save_format='bmp'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely


Found 55 images belonging to 5 classes.
(1, 100, 100, 3)


In [5]:
val_generator = ImageDataGenerator()
validation = val_generator.flow_from_directory('data/validation',target_size=(img_height,img_width),batch_size=32,class_mode="categorical")

Found 30 images belonging to 5 classes.


In [6]:
model = Sequential([
    BatchNormalization(axis=1, input_shape=(img_width,img_height,3)),
    Convolution2D(32, (3,3), activation='relu'),
    BatchNormalization(axis=1),
    MaxPooling2D(),
    Convolution2D(64, (3,3), activation='relu'),
    BatchNormalization(axis=1),
    MaxPooling2D(),
    Convolution2D(128, (3,3), activation='relu'),
    BatchNormalization(axis=1),
    MaxPooling2D(),
    Flatten(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(5, activation='sigmoid')
])
model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 100, 100, 3)       400       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 98, 98, 32)        896       
_________________________________________________________________
batch_normalization_2 (Batch (None, 98, 98, 32)        392       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 47, 47, 64)        18496     
_________________________________________________________________
batch_normalization_3 (Batch (None, 47, 47, 64)        188       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 23, 64)        0         
__________

Now we train the model

In [7]:
model.fit_generator(train,steps_per_epoch=train.samples//32,epochs=60,validation_data=validation,validation_steps=validation.samples)

Epoch 1/60
1/1 [==============================] - 6s 6s/step - loss: 2.2007 - acc: 0.1250 - val_loss: 1.7082 - val_acc: 0.2667
Epoch 2/60
1/1 [==============================] - 3s 3s/step - loss: 2.4561 - acc: 0.2174 - val_loss: 1.5009 - val_acc: 0.3667
Epoch 3/60
1/1 [==============================] - 3s 3s/step - loss: 1.9774 - acc: 0.2812 - val_loss: 1.3182 - val_acc: 0.4667
Epoch 4/60
1/1 [==============================] - 3s 3s/step - loss: 1.3712 - acc: 0.4783 - val_loss: 1.2367 - val_acc: 0.5000
Epoch 5/60
1/1 [==============================] - 3s 3s/step - loss: 1.2690 - acc: 0.4783 - val_loss: 1.1845 - val_acc: 0.6667
Epoch 6/60
1/1 [==============================] - 3s 3s/step - loss: 1.3878 - acc: 0.5000 - val_loss: 1.1283 - val_acc: 0.7333
Epoch 7/60
1/1 [==============================] - 3s 3s/step - loss: 1.2651 - acc: 0.4688 - val_loss: 1.0840 - val_acc: 0.7333
Epoch 8/60
1/1 [==============================] - 3s 3s/step - loss: 1.2441 - acc: 0.5217 - val_loss: 1.0386 - 

In [8]:
model.save_weights('augmented_result.h5')

In [9]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.kernel_approximation import RBFSampler
from sklearn import svm
from sklearn.feature_selection import SelectFromModel
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA

In [10]:
totalepochs=100
epoch=0
train = train_generator.flow_from_directory("Handgesturedata/",target_size=(img_height,img_width),batch_size=500,class_mode='sparse')
for i,j in enumerate(train):
    X = j[0]
    y= j[1]
    break;
counter=0
for i,j in enumerate(train):
    if counter > 10:
        break
    X = np.append(X, j[0],axis=0)
    y = np.append(y, j[1],axis=0)
    counter+=1

#use Sklearn algorithms here
X = X.reshape((len(X),-1))
print("Shape of data after flattening the image: ", X.shape)
print("Shape of the target value: ", y)

#Polynomial features expansion to degree 2
#poly = PolynomialFeatures(degree=2,interaction_only=True)
#X = poly.fit_transform(X)

#radial basis function kernel over the transformed data
#rbf_feature = RBFSampler(gamma=1, random_state=1)
#X = rbf_feature.fit_transform(X)


#feature selection using Lasso
lsvc = svm.LinearSVC(C=0.01).fit(X,y)
skmodel = SelectFromModel(lsvc, prefit=True)
#ard = ARDRegression()
#skmodel = SelectFromModel(ard, prefit=True)
X = skmodel.transform(X)

print("Shape of data after feature selection: ", X.shape)
#we intiate the classifier objects here
#decreasing the number of principal components to be lesser than the number of training examples.
pca = PCA(n_components=1000)
X=pca.fit_transform(X)

print("Shape of data after applying PCA: ", X.shape)

Found 85 images belonging to 5 classes.
Shape of data after flattening the image:  (1020, 30000)
Shape of the target value:  [1 2 3 ... 4 4 4]
Shape of data after feature selection:  (1020, 13434)
Shape of data after applying PCA:  (1020, 1000)


In [11]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
kf = KFold(n_splits=3)
kf.get_n_splits(X)
print(kf)
finalaccuracy=[]
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    xgmodel = XGBClassifier()
    xgmodel.fit(X_train, y_train)
    y_pred = xgmodel.predict(X_test)
    predictions = [round(value) for value in y_pred]
    accuracy = accuracy_score(y_test, predictions)
    print(accuracy*100.0)
    finalaccuracy.append(accuracy*100.0)
print("Accuracy: ", sum(finalaccuracy)/float(len(finalaccuracy)))

KFold(n_splits=3, random_state=None, shuffle=False)


C:\Users\vikyl\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


97.05882352941177


C:\Users\vikyl\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


98.23529411764706
97.6470588235294
Accuracy:  97.6470588235294


C:\Users\vikyl\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [12]:
from sklearn.ensemble import AdaBoostClassifier
finalaccuracy=[]
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y0_train, y_test = y[train_index], y[test_index]
    svmadamodel = AdaBoostClassifier(base_estimator=svm.SVC(probability=True, kernel="poly", degree=5), n_estimators=50)
    svmadamodel.fit(X_train, y_train)
    y_pred = svmadamodel.predict(X_test)
    predictions = [round(value) for value in y_pred]
    accuracy = accuracy_score(y_test, predictions)
    print(accuracy*100.0)
    finalaccuracy.append(accuracy*100.0)
print("Accuracy: ", sum(finalaccuracy)/float(len(finalaccuracy)))

23.235294117647058
70.0
96.76470588235294
Accuracy:  63.333333333333336


In [13]:
from sklearn.ensemble import RandomForestClassifier
finalaccuracy=[]
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    rfcmodel =  RandomForestClassifier(n_estimators=100)
    rfcmodel.fit(X_train, y_train)
    y_pred = rfcmodel.predict(X_test)
    predictions = [round(value) for value in y_pred]
    accuracy = accuracy_score(y_test, predictions)
    print(accuracy*100.0)
    finalaccuracy.append(accuracy*100.0)
print("Accuracy: ", sum(finalaccuracy)/float(len(finalaccuracy)))

93.23529411764706
90.58823529411765
90.29411764705883
Accuracy:  91.37254901960785


Here, we load the test pictures and calculate the accuracy.

In [14]:
path = ("test/")
img_width, img_height = 100, 100
batchsize=32
for subfolder in os.listdir(path):
    print(subfolder)
    for item in os.listdir(path+subfolder):
        #print(item)
        if item.endswith(".bmp"):
            im = Image.open(path+subfolder+"/"+item)
            print(path+subfolder+"/"+item)
            f, e = os.path.split(path+subfolder+"/"+item)
            imResize = im.resize((100,100), Image.ANTIALIAS)
            im.close()
            imResize.save(path+subfolder+"/"+item,"JPEG")

A
test/A/a01.bmp
test/A/a02.bmp
B
test/B/b01.bmp
test/B/b02.bmp
C
test/C/c01.bmp
test/C/c02.bmp
D
test/D/d01.bmp
test/D/d02.bmp
NA
test/NA/1.bmp
test/NA/2.bmp


In [15]:
epoch=0
test_generator = ImageDataGenerator()
test = test_generator.flow_from_directory("test/",target_size=(img_height,img_width),batch_size=10,class_mode='categorical')
for i,j in enumerate(test):
    X_test = j[0]
    y_test = j[1]
    print(X_test.shape)
    print(y_test.shape)
    break;


Found 10 images belonging to 5 classes.
(10, 100, 100, 3)
(10, 5)


Now we test the test data on the Neural Network.

In [16]:
print(model.metrics_names)
model.test_on_batch(X_test, y_test)

['loss', 'acc']


[0.60420686, 1.0]

In [17]:
epoch=0
test_generator = ImageDataGenerator()
test = test_generator.flow_from_directory("test/",target_size=(img_height,img_width),batch_size=10,class_mode='sparse')
for i,j in enumerate(test):
    X_test = j[0]
    y_test = j[1]
    break;

#use Sklearn algorithms here
X_test2 = X_test.reshape((len(X_test),-1))

#ard = ARDRegression()
#skmodel = SelectFromModel(ard, prefit=True)
X_test2 = skmodel.transform(X_test2)
#we intiate the classifier objects here
#decreasing the number of principal components to be lesser than the number of training examples.
#pca = PCA(n_components=1000)
X_test2=pca.transform(X_test2)


Found 10 images belonging to 5 classes.


Now, we test the data on non Neural Network based algorithms which were trained above.

In [18]:
#We test XGboost classifier on the test set
y_pred = xgmodel.predict(X_test2)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print(accuracy*100.0)

#We test SVM with adaboost on the test set
y_pred = svmadamodel.predict(X_test2)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print(accuracy*100.0)

#We train random forest classifier on the test set
y_pred = rfcmodel.predict(X_test2)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print(accuracy*100.0)

100.0
90.0
70.0


C:\Users\vikyl\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
